In [ ]:
# we will first record rows and colums as STRINGS
rows = 'ABCDEFGHI'
cols = '123456789'
assignments = []
# we will now define a helper function which when given 2 strings 'a' and 'b'
# will return a list of all possible concatenations of a letter 's' in 'a' with
# letter 't' in 'b'
def cross(a , b):
    return [s + t for s in a for t in b]

# creationg labels for boxes , row_units , col_units , square_units and peers
boxes = cross(rows , cols)
row_units = [cross(r , cols) for r in rows]
column_units = [cross(rows , c) for c in cols]
square_units = [cross(rs , cs) for rs in ('ABC' , 'DEF' ,'GHI' ) for cs in ('123' , '456' , '789')]
unitlist = row_units + column_units + square_units

units = dict((s , [u for u in unitlist if s in u])for s in boxes) # all the peers of the unit with the unit also included
peers = dict((s, set(sum(units[s] , [])) - set([s])) for s in boxes) # just the peers

def display(values):
    width = 1 + max(len(values[s]) for s in boxes)
    line = '+'.join(['-' *(width*3)]*3)
    for r in rows:
        print (''.join(values[r+c].center(width)+('|' if c in '36' else '') for c in cols))
        if r in 'CF':
            print(line)
    return

# converting the input string into a dictionary
def grid_values(grid):
    chars = []
    digits = '123456789'
    for c in grid:
        if c in digits:
            chars.append(c)
        if c == '.':
            chars.append(digits)
    assert len(chars) == 81
    return dict(zip(boxes, chars))

# STRATEGY_1 : eliminate values that are in the peers 
def eliminate(values):
    solved_values = [box for box in values.keys() if len(values[box]) == 1]
    for box in solved_values:
        digit = values[box]
        for peer in peers[box]:
            values[peer] = values[peer].replace(digit,'')
    return values

# STRATEGY_2 : if there is only one box in a unit which would allow a certain dight then allort that digit
def only_choice(values):
    for unit in unitlist:
        for digit in '123456789':
            dplaces = [box for box in unit if digit in values[box]]
            if len(dplaces) == 1:
                values[dplaces[0]] = digit
    return values

def reduce_puzzle(values):
    solved_values = [box for box in values.keys() if len(values[box]) == 1]
    stalled = False
    while not stalled:
        # boxes with already fixed values
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])
        
        # use eleminate strategy
        values = eliminate(values)
        # use only_choice strategy
        values = only_choice(values)
        
        # check how many boxes with new fixed values
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        # if no new values ditected delete the loop
        stalled = solved_values_before == solved_values_after
        
        # sanity check false witha box with 0 availabel values
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values

# STRATEGY_3 : pick a box with minimal no of possible values and try to solve each of the puzzles obtained 
#              by choosing each of these values recursively.
def search(values):
    values = reduce_puzzle(values)
    values = naked_twins(values)
    if values is False:
        return False # failed earlier
    if all(len(values[s]) == 1 for s in boxes): 
        return values ## Solved!
    n , s = min((len(values[s]), s) for s in boxes if len(values[s]) > 1)
    for value in values[s]:
        new_sudoku = values.copy()
        new_sudoku[s] = value
        attempt = search(new_sudoku)
        if attempt:
            return attempt

def assign_value(valuess , box , value):
    if values[box] == value:
        return values
    
    values[box] == value
    if len(value) == 1:
        assignments.append(valuess.copy())
    return values
        
# STRATEGY_4 : enforcing that no square outside the 2 naked twins contain the twin values
def naked_twins(values):
    possibilities = [box for box in values.keys() if len(values[box]) == 2]
    twins = [[box1 , box2] for box1 in possibilities for box2 in peers[box1] if set(values[box1])== set (values[box2])]
    
    #for b1 , b2 in twins:
     #   print(b1 , b2 , values[b1])
            
    for box1 , box2 in twins:
        peers1 = set(peers[box1])
        peers2 = set(peers[box2])
        
        peers_int = peers1.intersection(peers2)
        
        # delete the two digits from all common peers
        for peer_box in peers_int:
            for rm_val in values[box1]:
                values = assign_value(values , peer_box , values[peer_box].replace(rm_val,''))
    return values

    
grid2 = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
values = grid_values(grid2)
values = search(values)
display(values)